<a href="https://colab.research.google.com/github/hereagain-Y/TCR_VAE/blob/main/attentionWithdataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import random
import itertools
import numpy as np
import itertools
import pandas as pd 
import random
import time
from datetime import datetime
from pkgutil import extend_path
import numpy as np
import matplotlib.pyplot as plt

from six.moves import xrange
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
#
from scipy.spatial import distance
import random 
from scipy import spatial
from scipy import stats
from torch.utils.data import TensorDataset, DataLoader
import os
import glob

In [2]:
cuda = True
device = torch.device("cuda" if cuda else "cpu")
n_feature = 16   

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

def load_data(path,balance_sample_num):
    dat = pd.read_csv(path+"/latentdata.csv",sep=',')
    index, cnt =np.unique(dat['bags'],return_counts=True)
    data_list=[]
    for i in index:
        data_list.append(dat.loc[dat['bags']==i,'1':].to_numpy().tolist())
    max_length=np.max(cnt)
    data_list_max_length = []
    for i in range(2*balance_sample_num): # zero paddings
          #temp_pad = np.zeros((max_length - len(data_list[i]), 32)).tolist()
          temp_pad = np.zeros((max_length - len(data_list[i]), 32))
          temp_ins = data_list[i]
          temp_array = np.concatenate((temp_ins,temp_pad),axis=0)
          data_list_max_length.append(temp_array)
    seq_array = np.array(data_list_max_length)
    label=[0,1]
    labels=list(itertools.chain.from_iterable(itertools.repeat(x, balance_sample_num) for x in label))
    label_array=np.array(labels)

    data = torch.tensor(seq_array, dtype=torch.float32)
    target = torch.tensor(label_array,dtype=torch.float32)
    dataset_all = TensorDataset(data,target)

    return data_list,dataset_all

In [5]:
dat = pd.read_csv("/content/drive/My Drive/DL/latentdata.csv",sep=',')

In [6]:
data_list,dataset_all =load_data(r'/content/drive/My Drive/DL',9)

In [25]:
train_loader = DataLoader(dataset_all, batch_size= 10,shuffle=True)

In [27]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L =32
        self.K = 1
        self.fclayers=nn.Sequential(
            nn.Linear(self.L, self.L),
            nn.Dropout(0.5),
            nn.ReLU(), # nn,SELU()
            nn.Linear(self.L, self.L),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(self.L, self.L),
            nn.Dropout(0.5),
            nn.ReLU()
            
        )
        self.attention = nn.Linear(self.L,self.K)
   

        self.out = nn.Sequential(
            nn.Linear(self.L*self.K,self.L) ,
            nn.BatchNorm1d(self.L),
            nn.ReLU())
        
        self.classifier = nn.Sequential(
            nn.Linear(self.L * self.K, 1), 
            nn.Sigmoid()
        )
    
    def forward(self,x):
        out = self.fclayers(x) #32

        attention = torch.squeeze(self.attention(out)) # 1 
        attention = torch.transpose(attention,0,1)

     # attention =self.sparseatt(attention)
        attention =torch.softmax(attention,dim=1)
      #attention_bag= torch.softmax(attention,dim=1) 
      
        out = torch.transpose(out,0,1)
        new_features = torch.bmm(torch.unsqueeze(attention, 1), out)
        new_features = torch.squeeze(new_features)
      
        sample_feature =self.out(new_features)
        predictions = self.classifier(sample_feature) 
        Y_hat = torch.ge(predictions, 0.5).float() 
        return predictions, attention,Y_hat


     # attention = torch.transpose(attention,0,1)


      # * original to get new futures 





    

In [28]:
model = Attention().to(device)

In [29]:
def loss_function( label, prediction ):
    reproduction_loss = nn.functional.binary_cross_entropy(label, prediction, reduction='sum')
    return reproduction_loss
       
def calculate_classification_error(Y_hat,Y):
    Y = Y.float()
    error = 1. - Y_hat.eq(Y).cpu().float().mean().data

    return error, Y_hat  

In [30]:

from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=1e-3)  
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
epochs=1000

In [32]:
train_loss= []
for epoch in range(epochs):
    overall_loss =0
    train_error = 0 
    for batch_idx, (data, target) in enumerate(train_loader):
      data,target = torch.transpose(data,1,0).to(device),target.to(device) #[5]
      optimizer.zero_grad()

      predicitons, attention,Y_hat = model(data)
      print(attention)
      predicitons =torch.squeeze(predicitons)#[5]
      loss =loss_function(predicitons,target)
      auc = roc_auc_score(target.cpu().detach().numpy(),predicitons.cpu().detach().numpy())
      label =target.cpu().detach().numpy()
      y = Y_hat.cpu().detach().numpy()
      overall_loss += loss.item()
      #print(overall_loss)
      loss.backward()
      optimizer.step()
    train_loss.append(overall_loss/len(dataset_all))

    if (epoch % 100 == 0):
      print("\tEpoch", epoch , "complete!", "\tAverage Loss: ", train_loss[epoch],accuracy_score(label, y)*100,'auc:',auc)
    


Streaming output truncated to the last 5000 lines.
        [1.1396e-04, 4.1221e-04, 7.0775e-04,  ..., 1.2553e-04, 6.2074e-05,
         6.4524e-05],
        [4.0747e-04, 1.0891e-03, 8.1945e-04,  ..., 6.0075e-05, 5.9635e-05,
         1.0421e-04],
        [1.2449e-04, 2.9477e-04, 2.4801e-04,  ..., 3.7986e-05, 4.4152e-05,
         4.6484e-05]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([[1.0131e-04, 3.5415e-04, 3.6732e-04,  ..., 3.4697e-05, 1.7873e-05,
         2.0143e-05],
        [8.6438e-04, 1.1448e-04, 1.1390e-04,  ..., 5.5977e-05, 4.8042e-05,
         7.7320e-05],
        [3.3776e-04, 9.8715e-05, 2.7644e-04,  ..., 4.8654e-05, 3.8017e-05,
         4.4425e-05],
        ...,
        [5.3538e-05, 4.3893e-05, 7.3099e-04,  ..., 4.8866e-05, 3.9814e-05,
         4.2762e-05],
        [1.9570e-04, 1.8529e-03, 3.7145e-04,  ..., 9.8423e-05, 8.0315e-05,
         5.4677e-05],
        [6.6555e-04, 1.9844e-04, 1.5425e-04,  ..., 8.9803e-05, 9.0231e-05,
         1.0042e-04]], device='cuda:0',

KeyboardInterrupt: ignored